In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, concatenate
from tensorflow.keras.optimizers import Adam

# Set file paths
original_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw"
output_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done"
csv_file_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\REPORTS\gamma_values.csv"
input_image_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\SRFP DAILY REPORTS\SRFP FINAL REPORT\Original Image.png"

# Load gamma values from the CSV file
gamma_df = pd.read_csv(csv_file_path)
gamma_dict = dict(zip(gamma_df['file_name'], gamma_df['gamma_value']))

# Define function to load, resize, and preprocess images
def load_images(original_folder, output_folder, file_name):
    original_path = os.path.join(original_folder, file_name)
    enhanced_path = os.path.join(output_folder, file_name)

    original_img = cv2.imread(original_path)
    enhanced_img = cv2.imread(enhanced_path)

    # Resize images
    original_img = cv2.resize(original_img, (500, 500))
    enhanced_img = cv2.resize(enhanced_img, (500, 500))

    original_img = original_img / 255.0  # Normalize original image
    enhanced_img = enhanced_img / 255.0  # Normalize enhanced image

    return original_img, enhanced_img

# Create the training dataset
input_images = []
output_images = []
gammas = []

for file_name in os.listdir(original_folder):
    original_img, enhanced_img = load_images(original_folder, output_folder, file_name)
    gamma = gamma_dict.get(file_name, 1.0)  # Default gamma value is 1.0 if not found in the dictionary

    input_images.append(original_img)
    output_images.append(enhanced_img)
    gammas.append(gamma)

input_images = np.array(input_images)
output_images = np.array(output_images)
gammas = np.array(gammas)

# Define the model architecture
def build_model():
    input_shape = (500, 500, 3)
    input_image = Input(shape=input_shape, name='input_image')
    input_gamma = Input(shape=(1,), name='input_gamma')

    conv1 = Conv2D(32, 3, activation='relu', padding='same')(input_image)
    conv2 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
    conv3 = Conv2D(32, 3, activation='relu', padding='same')(conv2)

    conv_gamma = tf.repeat(input_gamma, tf.shape(conv3)[1]*tf.shape(conv3)[2])

    conv_gamma = tf.reshape(conv_gamma, (-1, tf.shape(conv3)[1], tf.shape(conv3)[2], 1))

    concat = concatenate([conv3, conv_gamma], axis=-1)

    output = Conv2D(3, 3, activation='sigmoid', padding='same')(concat)

    model = Model(inputs=[input_image, input_gamma], outputs=output)
    model.compile(optimizer=Adam(lr=0.1), loss='mse')

    return model



# Build and train the model
model = build_model()
model.fit([input_images, gammas], output_images, batch_size=16, epochs=15)

# Save the model weights
model.save_weights('model_weights.h5')

# Load the input image, resize, and preprocess it
input_img = cv2.imread(input_image_path)
input_img = cv2.resize(input_img, (500, 500))
input_img = input_img / 255.0  # Normalize input image
input_img = np.expand_dims(input_img, axis=0)  # Add batch dimension

# Get the optimal gamma value for the input image
input_file_name = os.path.basename(input_image_path)
optimal_gamma = gamma_dict.get(input_file_name, 1.0)  # Default gamma value is 1.0 if not found in the dictionary

# Generate the enhanced image
# Generate the enhanced image
gamma_input = np.array([[optimal_gamma]])  # Convert gamma input to NumPy array
enhanced_img = model.predict([input_img, gamma_input])
enhanced_img = enhanced_img[0] * 255.0  # Denormalize enhanced image
enhanced_img = enhanced_img.astype(np.uint8)


# Save the enhanced image
output_image_path = os.path.join(output_folder, input_file_name)
cv2.imwrite(output_image_path, enhanced_img)


Epoch 1/15
20/20 [==============================] - 259s 10s/step - loss: 0.0549
Epoch 2/15
20/20 [==============================] - 186s 9s/step - loss: 0.0256
Epoch 3/15
20/20 [==============================] - 185s 9s/step - loss: 0.0113
Epoch 4/15
20/20 [==============================] - 182s 9s/step - loss: 0.0096
Epoch 5/15
20/20 [==============================] - 190s 10s/step - loss: 0.0080
Epoch 6/15
20/20 [==============================] - 603s 31s/step - loss: 0.0079
Epoch 7/15
20/20 [==============================] - 188s 9s/step - loss: 0.0082
Epoch 8/15
20/20 [==============================] - 188s 9s/step - loss: 0.0081
Epoch 9/15
20/20 [==============================] - 185s 9s/step - loss: 0.0068
Epoch 10/15
20/20 [==============================] - 186s 9s/step - loss: 0.0076
Epoch 11/15
20/20 [==============================] - 227s 11s/step - loss: 0.0072
Epoch 12/15
20/20 [==============================] - 194s 10s/step - loss: 0.0070
Epoch 13/15
20/20 [=============

True

In [4]:
output_image_path

'C:\\Users\\MAHESH\\Desktop\\SRFP INTERNSHIP\\train_in_1\\Done\\Original Image.png'

In [13]:
print(enhanced_img)

[[[ 87  88  79]
  [ 70  76  67]
  [ 64  71  63]
  ...
  [ 72  79  70]
  [ 75  85  73]
  [100  98 103]]

 [[ 78  72  58]
  [ 53  49  42]
  [ 48  42  38]
  ...
  [ 62  55  49]
  [ 64  63  55]
  [ 89  80  87]]

 [[ 71  64  51]
  [ 46  40  34]
  [ 41  33  31]
  ...
  [ 57  46  42]
  [ 61  55  50]
  [ 84  74  85]]

 ...

 [[ 54  58  44]
  [ 26  31  26]
  [ 20  26  23]
  ...
  [ 19  34  36]
  [ 25  40  40]
  [ 56  63  80]]

 [[ 59  61  48]
  [ 31  37  29]
  [ 25  32  28]
  ...
  [ 24  38  38]
  [ 30  44  42]
  [ 60  69  82]]

 [[ 74  88  69]
  [ 46  66  50]
  [ 43  62  49]
  ...
  [ 40  64  55]
  [ 47  69  61]
  [ 72  88  88]]]


In [11]:
input_image_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\REPORTS\FNfo7r.jpg"

In [9]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, concatenate
from tensorflow.keras.optimizers import Adam

# Set file paths
original_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw"
output_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done"
csv_file_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\REPORTS\gamma_values.csv"
input_image_path = r"C:\Users\MAHESH\Pictures\ATIFR 2022\243.jpeg"

# Load gamma values from the CSV file
gamma_df = pd.read_csv(csv_file_path)
gamma_dict = dict(zip(gamma_df['file_name'], gamma_df['gamma_value']))

# Define function to load, resize, and preprocess images
def load_images(original_folder, output_folder, file_name):
    original_path = os.path.join(original_folder, file_name)
    enhanced_path = os.path.join(output_folder, file_name)

    original_img = cv2.imread(original_path)
    enhanced_img = cv2.imread(enhanced_path)

    # Resize images
    original_img = cv2.resize(original_img, (500, 500))
    enhanced_img = cv2.resize(enhanced_img, (500, 500))

    original_img = original_img / 255.0  # Normalize original image
    enhanced_img = enhanced_img / 255.0  # Normalize enhanced image

    return original_img, enhanced_img

# Create the training dataset
input_images = []
output_images = []
gammas = []

for file_name in os.listdir(original_folder):
    original_img, enhanced_img = load_images(original_folder, output_folder, file_name)
    gamma = gamma_dict.get(file_name, 1.0)  # Default gamma value is 1.0 if not found in the dictionary

    input_images.append(original_img)
    output_images.append(enhanced_img)
    gammas.append(gamma)

input_images = np.array(input_images)
output_images = np.array(output_images)
gammas = np.array(gammas)

# Define the model architecture
def build_model():
    input_shape = (500, 500, 3)
    input_image = Input(shape=input_shape, name='input_image')
    input_gamma = Input(shape=(1,), name='input_gamma')

    conv1 = Conv2D(32, 3, activation='relu', padding='same')(input_image)
    conv2 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
    conv3 = Conv2D(32, 3, activation='relu', padding='same')(conv2)

    conv_gamma = tf.repeat(input_gamma, tf.shape(conv3)[1]*tf.shape(conv3)[2])

    conv_gamma = tf.reshape(conv_gamma, (-1, tf.shape(conv3)[1], tf.shape(conv3)[2], 1))

    concat = concatenate([conv3, conv_gamma], axis=-1)

    output = Conv2D(3, 3, activation='sigmoid', padding='same')(concat)

    model = Model(inputs=[input_image, input_gamma], outputs=output)
    model.compile(optimizer=Adam(lr=0.1), loss='mse')

    return model

# Build and train the model
model = build_model()
model.fit([input_images, gammas], output_images, batch_size=8, epochs=80)

# Save the model weights
model.save_weights('model_weights.h5')

# Load the input image, resize, and preprocess it
input_img = cv2.imread(input_image_path)
input_img = cv2.resize(input_img, (500, 500))
input_img = input_img / 255.0  # Normalize input image
input_img = np.expand_dims(input_img, axis=0)  # Add batch dimension

# Get the optimal gamma value for the input image
input_file_name = os.path.basename(input_image_path)
optimal_gamma = gamma_dict.get(input_file_name, 1.0)  # Default gamma value is 1.0 if not found in the dictionary

# Generate the enhanced image
gamma_input = np.array([[optimal_gamma]])  # Convert gamma input to NumPy array
enhanced_img = model.predict([input_img, gamma_input])
enhanced_img = enhanced_img[0] * 255.0  # Denormalize enhanced image
enhanced_img = enhanced_img.astype(np.uint8)

# Save the enhanced image
output_image_path = os.path.join(output_folder, input_file_name)
cv2.imwrite(output_image_path, enhanced_img)


Epoch 1/80
39/39 [==============================] - 193s 5s/step - loss: 0.0469
Epoch 2/80
39/39 [==============================] - 191s 5s/step - loss: 0.0118
Epoch 3/80
39/39 [==============================] - 4589s 121s/step - loss: 0.0080
Epoch 4/80
39/39 [==============================] - 187s 5s/step - loss: 0.0082
Epoch 5/80
39/39 [==============================] - 187s 5s/step - loss: 0.0068
Epoch 6/80
39/39 [==============================] - 183s 5s/step - loss: 0.0066
Epoch 7/80
39/39 [==============================] - 184s 5s/step - loss: 0.0073
Epoch 8/80
39/39 [==============================] - 185s 5s/step - loss: 0.0070
Epoch 9/80
39/39 [==============================] - 186s 5s/step - loss: 0.0083
Epoch 10/80
39/39 [==============================] - 5758s 151s/step - loss: 0.0066
Epoch 11/80
39/39 [==============================] - 184s 5s/step - loss: 0.0058
Epoch 12/80
39/39 [==============================] - 182s 5s/step - loss: 0.0060
Epoch 13/80
39/39 [============

1/1 [==============================] - 0s 357ms/step


True

In [10]:
output_image_path

'C:\\Users\\MAHESH\\Desktop\\SRFP INTERNSHIP\\train_in_1\\Done\\243.jpeg'

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, concatenate
from tensorflow.keras.optimizers import Adam

# Set file paths
original_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Raw"
output_folder = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\train_in_1\Done"
csv_file_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\REPORTS\gamma_values.csv"
input_image_path = r"C:\Users\MAHESH\Desktop\SRFP INTERNSHIP\SRFP DAILY REPORTS\SRFP FINAL REPORT\Man-doing-postproduction-of-his-photos-on-laptop-at-night-By-Carina-Konig.jpg"

# Load gamma values from the CSV file
gamma_df = pd.read_csv(csv_file_path)
gamma_dict = dict(zip(gamma_df['file_name'], gamma_df['gamma_value']))

# Define function to load, resize, and preprocess images
def load_images(original_folder, output_folder, file_name):
    original_path = os.path.join(original_folder, file_name)
    enhanced_path = os.path.join(output_folder, file_name)

    original_img = cv2.imread(original_path)
    enhanced_img = cv2.imread(enhanced_path)

    # Resize images
    original_img = cv2.resize(original_img, (500, 500))
    enhanced_img = cv2.resize(enhanced_img, (500, 500))

    original_img = original_img / 255.0  # Normalize original image
    enhanced_img = enhanced_img / 255.0  # Normalize enhanced image

    return original_img, enhanced_img

# Create the training dataset
input_images = []
output_images = []
gammas = []

for file_name in os.listdir(original_folder):
    original_img, enhanced_img = load_images(original_folder, output_folder, file_name)
    gamma = gamma_dict.get(file_name, 1.0)  # Default gamma value is 1.0 if not found in the dictionary

    input_images.append(original_img)
    output_images.append(enhanced_img)
    gammas.append(gamma)

input_images = np.array(input_images)
output_images = np.array(output_images)
gammas = np.array(gammas)

# Define the model architecture
def build_model():
    input_shape = (500, 500, 3)
    input_image = Input(shape=input_shape, name='input_image')
    input_gamma = Input(shape=(1,), name='input_gamma')

    conv1 = Conv2D(32, 3, activation='linear', padding='same')(input_image)
    conv2 = Conv2D(32, 3, activation='linear', padding='same')(conv1)
    conv3 = Conv2D(32, 3, activation='linear', padding='same')(conv2)

    conv_gamma = tf.repeat(input_gamma, tf.shape(conv3)[1]*tf.shape(conv3)[2])

    conv_gamma = tf.reshape(conv_gamma, (-1, tf.shape(conv3)[1], tf.shape(conv3)[2], 1))

    concat = concatenate([conv3, conv_gamma], axis=-1)

    output = Conv2D(3, 3, activation='sigmoid', padding='same')(concat)

    model = Model(inputs=[input_image, input_gamma], outputs=output)
    model.compile(optimizer=Adam(lr=0.1), loss='mse')

    return model

# Build and train the model
model = build_model()
model.fit([input_images, gammas], output_images, batch_size=16, epochs=10)

# Save the model weights
model.save_weights('model_weights.h5')

# Load the input image, resize, and preprocess it
input_img = cv2.imread(input_image_path)
input_img = cv2.resize(input_img, (500, 500))
input_img = input_img / 255.0  # Normalize input image
input_img = np.expand_dims(input_img, axis=0)  # Add batch dimension

# Get the optimal gamma value for the input image
input_file_name = os.path.basename(input_image_path)
optimal_gamma = gamma_dict.get(input_file_name, 1.0)  # Default gamma value is 1.0 if not found in the dictionary

# Generate the enhanced image
gamma_input = np.array([[optimal_gamma]])  # Convert gamma input to NumPy array
enhanced_img = model.predict([input_img, gamma_input])
enhanced_img = enhanced_img[0] * 255.0  # Denormalize enhanced image
enhanced_img = enhanced_img.astype(np.uint8)

# Save the enhanced image
output_image_path = os.path.join(output_folder, input_file_name)
cv2.imwrite(output_image_path, enhanced_img)


Epoch 1/10
20/20 [==============================] - 211s 9s/step - loss: 0.0447
Epoch 2/10
20/20 [==============================] - 179s 9s/step - loss: 0.0220
Epoch 3/10
20/20 [==============================] - 176s 9s/step - loss: 0.0124
Epoch 4/10
20/20 [==============================] - 177s 9s/step - loss: 0.0087
Epoch 5/10
20/20 [==============================] - 178s 9s/step - loss: 0.0081
Epoch 6/10
20/20 [==============================] - 177s 9s/step - loss: 0.0077
Epoch 7/10
20/20 [==============================] - 177s 9s/step - loss: 0.0080
Epoch 8/10
20/20 [==============================] - 192s 10s/step - loss: 0.0079
Epoch 9/10
20/20 [==============================] - 201s 10s/step - loss: 0.0074
Epoch 10/10
1/1 [==============================] - 0s 468ms/step


True

In [11]:
output_image_path

'C:\\Users\\MAHESH\\Desktop\\SRFP INTERNSHIP\\train_in_1\\Done\\243.jpeg'